Import pandas, numpy.

Install lxml to retrieve data from html

In [2]:
import pandas as pd
import numpy as np
!conda install lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h7b6447c_1         3.8 MB
    certifi-2019.6.16          |           py36_1         156 KB
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1     conda-forge --> 2019.6.16-py36_1    
    lxml:    4.2.5-py37hefd8a0e_0             --> 4.3.0-py36hefd8a0e_0
    openssl: 1.1.1

Import lxml. Read Wikipedia html page.

In [3]:
import lxml

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs=pd.read_html(url)
dfs

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns],
                                                   0   \
 0                                                NaN   
 1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
 2                                                 NL   
 3                                                  A   
 
                          

Transform to a dataframe, named 'TOR_zip'. Then rename the columns.

In [5]:
TOR_zip=pd.DataFrame(dfs[0])
TOR_zip

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [6]:
#rename columns
TOR_zip.columns=['PostalCode','Borough','Neighborhood']
TOR_zip.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop cells with a Borough that is Not Assigned. Pull a list of the boroughs to make sure that is no other weird names to clean up.

In [7]:
TOR_zip=TOR_zip[TOR_zip['Borough']!='Not assigned']
TOR_zip['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

Check if there is any neighborhood that is Not Assigned.

In [8]:
TOR_zip[TOR_zip['Neighborhood']=='Not Assigned']

,PostalCode,Borough,Neighborhood


Count the number of unique postal codes. After cleaning up the table, total rows should match the number of postal codes.

In [9]:
TOR_zip.groupby('PostalCode').count()

,Borough,Neighborhood
PostalCode,,
M1B,2,2
M1C,3,3
M1E,3,3
M1G,1,1
M1H,1,1
...,...,...
M9N,1,1
M9P,1,1
M9R,4,4


Create TOR_zip_n dataframe for unique postal codes. Postal codes that cover multiple neighborhoods will be combined to one line. 

In [10]:
TOR_zip.sort_values('PostalCode',ascending=True,inplace=True)
TOR_zip_n=TOR_zip.head(1)

for i in range(1,TOR_zip.shape[0]):
    if TOR_zip.iloc[i,0]==TOR_zip.iloc[i-1,0]:
        if TOR_zip.iloc[i,1]==TOR_zip.iloc[i,1]:
            TOR_zip_n.iloc[-1,2]=TOR_zip_n.iloc[-1,2]+', '+TOR_zip.iloc[i,2]
        else:
            new_df=pd.DataFrame(TOR_zip.iloc[i,:]).T
            TOR_zip_n=pd.concat([TOR_zip_n,new_df],axis=0)
    else:
        new_df=pd.DataFrame(TOR_zip.iloc[i,:]).T
        TOR_zip_n=pd.concat([TOR_zip_n,new_df],axis=0)
        
TOR_zip_n.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

,PostalCode,Borough,Neighborhood
11,M1B,Scarborough,"Rouge, Malvern"
29,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
42,M1E,Scarborough,"Guildwood, Morningside, West Hill"
53,M1G,Scarborough,Woburn
62,M1H,Scarborough,Cedarbrae


Count rows and columns of the unique postal code dataframe.

In [11]:
TOR_zip_n.shape

(103, 3)

Retrieve Latitude and Longitude from GeoCoder

In [12]:
!pip install geocoder
import geocoder # import geocoder


     |████████████████████████████████| 102kB 2.1MB/s ta 0:00:011
     |████████████████████████████████| 829kB 24.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 12.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


      

Tried to retrieve data from GeoCoder but failed. Keyboard interrupted.

In [ ]:
postal_code=TOR_zip_n['PostalCode'].tolist()

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

Get geocoder from csv file.

In [13]:
geocode=pd.read_csv('https://cocl.us/Geospatial_data')
geocode

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Concatenate tables and reset index.

In [14]:
TOR_zip_n.set_index('PostalCode',inplace=True)
geocode.set_index('Postal Code',inplace=True)
TOR_zip_n=pd.concat([TOR_zip_n,geocode],axis=1)
TOR_zip_n.reset_index(inplace=True)
TOR_zip_n.rename(columns={'index':'PostalCode'},inplace=True)
TOR_zip_n.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
TOR_zip.to_csv('Toronto Neighborhoods.csv')